In [1]:
import pandas as pd
from openpyxl import Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import Font, Alignment, PatternFill, Border, Side

In [2]:
def remove(x):
    x.remove('')
    return x

def groupby_number(df, field):
    df_ = df.groupby('Part Number', as_index=False, sort=False).agg({field:list})
    df_[field] = df_[field].apply(lambda x: ';'.join(sorted(list(set(x)))))
    df_[field] = df_[field].apply(lambda x: sorted(list(set(x.split(';')))))
    df_[field] = df_[field].apply(lambda x: remove(x) if '' in x else x)
    return df_

def list_cleand(lst):
    new_list = []
    lst = list(set(lst))
    lst.sort()
    for x, y in zip(lst, lst[1:]):
        if int(y) - int(x) > 1:
            new_list.append(lst[:lst.index(y)])
            lst = lst[lst.index(y):]
    new_list.append(lst)
    return new_list

def handle_year(x):
    if len(x) > 1:
        return str(min(x)) + '-' + str(max(x))
    elif len(x) == 0:
        return ''
    else:
        return x[0]

def Vehicle(df):
    df['Make_Model'] = df['Make'] + ' ' + df['Model']
    df_MM_Y = df.groupby(['Part Number', 'Make_Model'], as_index=False, sort=False).agg({'Year': list})
    df_MM_Y['Year'] = df_MM_Y['Year'].apply(lambda x: list_cleand(x))
    df_MM_Y = df_MM_Y.explode('Year').fillna('')
    df_MM_Y['Year'] = df_MM_Y['Year'].apply(lambda x: handle_year(x))
    df_MM_Y['Vehicle'] = df_MM_Y['Make_Model'] + ' ' + df_MM_Y['Year']
    df_ = df_MM_Y.groupby('Part Number', as_index=False, sort=False).agg({'Vehicle': list})
    df_['Vehicle'] = df_['Vehicle'].apply(lambda x: '\n'.join(x))
    return df_

In [3]:
df_input = pd.read_excel('./模板_1.xlsx', dtype=str)
df_input['去重字段'] = df_input['Part Number']  + df_input['Year'] + df_input['Make'] + df_input['Model']
df_input.drop_duplicates(subset=['去重字段'],keep='last',inplace=True)

df_input['M_M_Y'] = df_input['Make'] + ' ' + df_input['Model'] + ' ' + df_input['Year']
df_input['M_M_Y'] = df_input['M_M_Y'].apply(lambda x: str(x).title())
df_repeat_MMY = df_input.groupby('M_M_Y', as_index=False, sort=False).agg({'Part Number': list})
df_repeat_MMY['重复号码'] = df_repeat_MMY['Part Number'].apply(lambda x: ';'.join(sorted(list(set(x)), key=x.index)) if len(x) > 1 else '')
df_repeat_number = pd.merge(df_input, df_repeat_MMY.drop(['Part Number'], axis=1), on='M_M_Y', how='left').fillna('')
df_repeat_number

,Part Number,Year,Make,Model,去重字段,M_M_Y,重复号码
0,YT-TPMS001,2007,Buick,Allure,YT-TPMS0012007BuickAllure,Buick Allure 2007,YT-TPMS001;YT-TPMS010
1,YT-TPMS001,2008,Buick,Allure,YT-TPMS0012008BuickAllure,Buick Allure 2008,YT-TPMS001;YT-TPMS010
2,YT-TPMS001,2008,Buick,Enclave,YT-TPMS0012008BuickEnclave,Buick Enclave 2008,YT-TPMS001;YT-TPMS010
3,YT-TPMS001,2009,Buick,Enclave,YT-TPMS0012009BuickEnclave,Buick Enclave 2009,YT-TPMS001;YT-TPMS006
4,YT-TPMS001,2010,Buick,Enclave,YT-TPMS0012010BuickEnclave,Buick Enclave 2010,YT-TPMS001;YT-TPMS006
...,...,...,...,...,...,...,...
9007,YT-TPMS255,2019,Hyundai,Kona Electric,YT-TPMS2552019HyundaiKona Electric,Hyundai Kona Electric 2019,
9008,YT-TPMS255,2020,Hyundai,Kona Electric,YT-TPMS2552020HyundaiKona Electric,Hyundai Kona Electric 2020,
9009,YT-TPMS255,2021,Hyundai,Kona Electric,YT-TPMS2552021HyundaiKona Electric,Hyundai Kona Electric 2021,
9010,YT-TPMS255,2022,Hyundai,Kona Electric,YT-TPMS2552022HyundaiKona Electric,Hyundai Kona Electric 2022,


In [4]:
df_all_repeat_number = groupby_number(df_repeat_number, '重复号码')
df_repeat_number_explode = df_all_repeat_number.explode('重复号码').fillna('')
df_all_repeat_number['重复号码'] = df_all_repeat_number['重复号码'].apply(lambda x: ';'.join(x) if x != None else '')
df_all_repeat_number

,Part Number,重复号码
0,YT-TPMS001,YT-TPMS001;YT-TPMS006;YT-TPMS008;YT-TPMS009;YT...
1,YT-TPMS002,YT-TPMS002;YT-TPMS007;YT-TPMS011;YT-TPMS018;YT...
2,YT-TPMS003,YT-TPMS003;YT-TPMS014;YT-TPMS020;YT-TPMS076;YT...
3,YT-TPMS004,YT-TPMS004;YT-TPMS005;YT-TPMS022;YT-TPMS023;YT...
4,YT-TPMS005,YT-TPMS004;YT-TPMS005;YT-TPMS021;YT-TPMS030;YT...
...,...,...
250,YT-TPMS251,
251,YT-TPMS252,YT-TPMS252;YT-TPMS254
252,YT-TPMS253,
253,YT-TPMS254,YT-TPMS252;YT-TPMS254


In [5]:
df_find_number = pd.merge(df_repeat_number_explode, df_repeat_number_explode, left_on='重复号码', right_on='Part Number', how='left').fillna('')
df_find_number['Part Number'] = df_find_number['Part Number_x']
df_find_number['完整重复号码'] = df_find_number['重复号码_x'] + ';' + df_find_number['重复号码_y']
df_find_number

,Part Number_x,重复号码_x,Part Number_y,重复号码_y,Part Number,完整重复号码
0,YT-TPMS001,YT-TPMS001,YT-TPMS001,YT-TPMS001,YT-TPMS001,YT-TPMS001;YT-TPMS001
1,YT-TPMS001,YT-TPMS001,YT-TPMS001,YT-TPMS006,YT-TPMS001,YT-TPMS001;YT-TPMS006
2,YT-TPMS001,YT-TPMS001,YT-TPMS001,YT-TPMS008,YT-TPMS001,YT-TPMS001;YT-TPMS008
3,YT-TPMS001,YT-TPMS001,YT-TPMS001,YT-TPMS009,YT-TPMS001,YT-TPMS001;YT-TPMS009
4,YT-TPMS001,YT-TPMS001,YT-TPMS001,YT-TPMS010,YT-TPMS001,YT-TPMS001;YT-TPMS010
...,...,...,...,...,...,...
7612,YT-TPMS255,YT-TPMS095,YT-TPMS095,YT-TPMS095,YT-TPMS255,YT-TPMS095;YT-TPMS095
7613,YT-TPMS255,YT-TPMS095,YT-TPMS095,YT-TPMS255,YT-TPMS255,YT-TPMS095;YT-TPMS255
7614,YT-TPMS255,YT-TPMS255,YT-TPMS255,YT-TPMS038,YT-TPMS255,YT-TPMS255;YT-TPMS038
7615,YT-TPMS255,YT-TPMS255,YT-TPMS255,YT-TPMS095,YT-TPMS255,YT-TPMS255;YT-TPMS095


In [6]:
df_table_all_repeat = df_find_number[['Part Number', '完整重复号码']]
df_table_all_repeat = df_table_all_repeat[df_table_all_repeat['完整重复号码'] != ';']
df_table_all_repeat = groupby_number(df_table_all_repeat, '完整重复号码')
df_table_all_repeat['完整重复号码'] = df_table_all_repeat['完整重复号码'].apply(lambda x: ';'.join(x) if x != None else '')
df_table_all_repeat

,Part Number,完整重复号码
0,YT-TPMS001,YT-TPMS001;YT-TPMS006;YT-TPMS008;YT-TPMS009;YT...
1,YT-TPMS002,YT-TPMS002;YT-TPMS007;YT-TPMS011;YT-TPMS017;YT...
2,YT-TPMS003,YT-TPMS003;YT-TPMS014;YT-TPMS020;YT-TPMS029;YT...
3,YT-TPMS004,YT-TPMS004;YT-TPMS005;YT-TPMS021;YT-TPMS022;YT...
4,YT-TPMS005,YT-TPMS004;YT-TPMS005;YT-TPMS021;YT-TPMS022;YT...
...,...,...
202,YT-TPMS249,YT-TPMS001;YT-TPMS006;YT-TPMS008;YT-TPMS009;YT...
203,YT-TPMS250,YT-TPMS020;YT-TPMS029;YT-TPMS094;YT-TPMS098;YT...
204,YT-TPMS252,YT-TPMS252;YT-TPMS254
205,YT-TPMS254,YT-TPMS252;YT-TPMS254


In [7]:
df_all_repeat_number_2 = groupby_number(df_table_all_repeat, '完整重复号码')
df_repeat_number_explode_2 = df_all_repeat_number_2.explode('完整重复号码').fillna('')
df_find_number_2 = pd.merge(df_repeat_number_explode_2, df_repeat_number_explode_2, left_on='完整重复号码', right_on='Part Number', how='left').fillna('')
df_find_number_2['Part Number'] = df_find_number_2['Part Number_x']
df_find_number_2['完整重复号码'] = df_find_number_2['完整重复号码_x'] + ';' + df_find_number_2['完整重复号码_y']
df_find_number_2

,Part Number_x,完整重复号码_x,Part Number_y,完整重复号码_y,Part Number,完整重复号码
0,YT-TPMS001,YT-TPMS001,YT-TPMS001,YT-TPMS001,YT-TPMS001,YT-TPMS001;YT-TPMS001
1,YT-TPMS001,YT-TPMS001,YT-TPMS001,YT-TPMS006,YT-TPMS001,YT-TPMS001;YT-TPMS006
2,YT-TPMS001,YT-TPMS001,YT-TPMS001,YT-TPMS008,YT-TPMS001,YT-TPMS001;YT-TPMS008
3,YT-TPMS001,YT-TPMS001,YT-TPMS001,YT-TPMS009,YT-TPMS001,YT-TPMS001;YT-TPMS009
4,YT-TPMS001,YT-TPMS001,YT-TPMS001,YT-TPMS010,YT-TPMS001,YT-TPMS001;YT-TPMS010
...,...,...,...,...,...,...
76104,YT-TPMS255,YT-TPMS255,YT-TPMS255,YT-TPMS203,YT-TPMS255,YT-TPMS255;YT-TPMS203
76105,YT-TPMS255,YT-TPMS255,YT-TPMS255,YT-TPMS229,YT-TPMS255,YT-TPMS255;YT-TPMS229
76106,YT-TPMS255,YT-TPMS255,YT-TPMS255,YT-TPMS236,YT-TPMS255,YT-TPMS255;YT-TPMS236
76107,YT-TPMS255,YT-TPMS255,YT-TPMS255,YT-TPMS240,YT-TPMS255,YT-TPMS255;YT-TPMS240


In [8]:
df_table_all_repeat_2 = df_find_number_2[['Part Number', '完整重复号码']]
df_table_all_repeat_2 = df_table_all_repeat_2[df_table_all_repeat_2['完整重复号码'] != ';']
df_table_all_repeat_2 = groupby_number(df_table_all_repeat_2, '完整重复号码')
df_table_all_repeat_2['完整重复号码'] = df_table_all_repeat_2['完整重复号码'].apply(lambda x: ';'.join(x) if x != None else '')
df_table_all_repeat_2

,Part Number,完整重复号码
0,YT-TPMS001,YT-TPMS001;YT-TPMS006;YT-TPMS008;YT-TPMS009;YT...
1,YT-TPMS002,YT-TPMS002;YT-TPMS003;YT-TPMS007;YT-TPMS011;YT...
2,YT-TPMS003,YT-TPMS002;YT-TPMS003;YT-TPMS011;YT-TPMS013;YT...
3,YT-TPMS004,YT-TPMS004;YT-TPMS005;YT-TPMS021;YT-TPMS022;YT...
4,YT-TPMS005,YT-TPMS004;YT-TPMS005;YT-TPMS021;YT-TPMS022;YT...
...,...,...
202,YT-TPMS249,YT-TPMS001;YT-TPMS006;YT-TPMS008;YT-TPMS009;YT...
203,YT-TPMS250,YT-TPMS003;YT-TPMS014;YT-TPMS020;YT-TPMS029;YT...
204,YT-TPMS252,YT-TPMS252;YT-TPMS254
205,YT-TPMS254,YT-TPMS252;YT-TPMS254


In [9]:
df_repeat = pd.merge(df_all_repeat_number, df_table_all_repeat_2, on='Part Number', how='left').fillna('')
df_repeat

,Part Number,重复号码,完整重复号码
0,YT-TPMS001,YT-TPMS001;YT-TPMS006;YT-TPMS008;YT-TPMS009;YT...,YT-TPMS001;YT-TPMS006;YT-TPMS008;YT-TPMS009;YT...
1,YT-TPMS002,YT-TPMS002;YT-TPMS007;YT-TPMS011;YT-TPMS018;YT...,YT-TPMS002;YT-TPMS003;YT-TPMS007;YT-TPMS011;YT...
2,YT-TPMS003,YT-TPMS003;YT-TPMS014;YT-TPMS020;YT-TPMS076;YT...,YT-TPMS002;YT-TPMS003;YT-TPMS011;YT-TPMS013;YT...
3,YT-TPMS004,YT-TPMS004;YT-TPMS005;YT-TPMS022;YT-TPMS023;YT...,YT-TPMS004;YT-TPMS005;YT-TPMS021;YT-TPMS022;YT...
4,YT-TPMS005,YT-TPMS004;YT-TPMS005;YT-TPMS021;YT-TPMS030;YT...,YT-TPMS004;YT-TPMS005;YT-TPMS021;YT-TPMS022;YT...
...,...,...,...
250,YT-TPMS251,,
251,YT-TPMS252,YT-TPMS252;YT-TPMS254,YT-TPMS252;YT-TPMS254
252,YT-TPMS253,,
253,YT-TPMS254,YT-TPMS252;YT-TPMS254,YT-TPMS252;YT-TPMS254


In [10]:
df_vehicle = Vehicle(df_input)
df_result_1 = pd.merge(df_vehicle, df_repeat, on='Part Number', how='left').fillna('')
df_result_1

,Part Number,Vehicle,重复号码,完整重复号码
0,YT-TPMS001,Buick Allure 2007-2008\nBuick Enclave 2008-201...,YT-TPMS001;YT-TPMS006;YT-TPMS008;YT-TPMS009;YT...,YT-TPMS001;YT-TPMS006;YT-TPMS008;YT-TPMS009;YT...
1,YT-TPMS002,Lexus CT200H 2011-2017\nLexus Gs300 2005-2006\...,YT-TPMS002;YT-TPMS007;YT-TPMS011;YT-TPMS018;YT...,YT-TPMS002;YT-TPMS003;YT-TPMS007;YT-TPMS011;YT...
2,YT-TPMS003,Chrysler 300 2010-2021\nDodge Challenger 2010-...,YT-TPMS003;YT-TPMS014;YT-TPMS020;YT-TPMS076;YT...,YT-TPMS002;YT-TPMS003;YT-TPMS011;YT-TPMS013;YT...
3,YT-TPMS004,Ford C-Max 2013-2018\nFord E-150 2010-2014\nFo...,YT-TPMS004;YT-TPMS005;YT-TPMS022;YT-TPMS023;YT...,YT-TPMS004;YT-TPMS005;YT-TPMS021;YT-TPMS022;YT...
4,YT-TPMS005,Ford Bronco 2021-2023\nFord Edge 2015-2023\nFo...,YT-TPMS004;YT-TPMS005;YT-TPMS021;YT-TPMS030;YT...,YT-TPMS004;YT-TPMS005;YT-TPMS021;YT-TPMS022;YT...
...,...,...,...,...
250,YT-TPMS251,Ram Promaster 1500 2022-2024\nRam Promaster 25...,,
251,YT-TPMS252,Nissan Kicks 2021-2024\nNissan Sentra 2020-202...,YT-TPMS252;YT-TPMS254,YT-TPMS252;YT-TPMS254
252,YT-TPMS253,Nissan Ariya 2023-2024\nNissan Rogue 2021-2023,,
253,YT-TPMS254,Nissan Kicks 2022\nNissan Sentra 2020-2023,YT-TPMS252;YT-TPMS254,YT-TPMS252;YT-TPMS254


In [11]:
df_repeat_vehicle = df_repeat_number[df_repeat_number['重复号码'] != '']
df_repeat_vehicle = df_repeat_vehicle[['重复号码', 'Year', 'Make', 'Model']]
df_repeat_vehicle

,重复号码,Year,Make,Model
0,YT-TPMS001;YT-TPMS010,2007,Buick,Allure
1,YT-TPMS001;YT-TPMS010,2008,Buick,Allure
2,YT-TPMS001;YT-TPMS010,2008,Buick,Enclave
3,YT-TPMS001;YT-TPMS006,2009,Buick,Enclave
4,YT-TPMS001;YT-TPMS006,2010,Buick,Enclave
...,...,...,...,...
9000,YT-TPMS038;YT-TPMS255,2019,Hyundai,Elantra
9001,YT-TPMS038;YT-TPMS255,2020,Hyundai,Elantra
9002,YT-TPMS038;YT-TPMS255,2018,Hyundai,Elantra GT
9003,YT-TPMS038;YT-TPMS255,2019,Hyundai,Elantra GT


In [12]:
df_repeat_vehicle.rename(columns={'重复号码':'Part Number'}, inplace=True)
df_repeat_vehicle_groupby = Vehicle(df_repeat_vehicle)
df_repeat_vehicle_groupby['重复车型'] = df_repeat_vehicle_groupby['Part Number'] + ':\n' + df_repeat_vehicle_groupby['Vehicle']
df_repeat_vehicle_groupby['重复车型'] = df_repeat_vehicle_groupby['重复车型'].apply(lambda x: x.replace('\n', '\n    '))
df_repeat_vehicle_groupby['Part Number'] = df_repeat_vehicle_groupby['Part Number'].apply(lambda x: x.split(';'))
df_repeat_vehicle_groupby

,Part Number,Vehicle,重复车型
0,"[YT-TPMS001, YT-TPMS010]",Buick Allure 2007-2008\nBuick Enclave 2008\nBu...,YT-TPMS001;YT-TPMS010:\n Buick Allure 2007-...
1,"[YT-TPMS001, YT-TPMS006]",Buick Enclave 2009-2012\nBuick Encore 2013-201...,YT-TPMS001;YT-TPMS006:\n Buick Enclave 2009...
2,"[YT-TPMS001, YT-TPMS006, YT-TPMS009]",Buick Enclave 2013-2015\nCadillac Escalade 201...,YT-TPMS001;YT-TPMS006;YT-TPMS009:\n Buick E...
3,"[YT-TPMS001, YT-TPMS006, YT-TPMS033]",Buick Regal 2013\nChevrolet Malibu 2013-2015,YT-TPMS001;YT-TPMS006;YT-TPMS033:\n Buick R...
4,"[YT-TPMS001, YT-TPMS009, YT-TPMS010, YT-TPMS01...",Cadillac CTS 2009,YT-TPMS001;YT-TPMS009;YT-TPMS010;YT-TPMS015;YT...
...,...,...,...
317,"[YT-TPMS236, YT-TPMS239]",Kia Optima 2012-2015\nKia Optima 2021,YT-TPMS236;YT-TPMS239:\n Kia Optima 2012-20...
318,"[YT-TPMS236, YT-TPMS242]",Suzuki Jimny 1998-2022\nSuzuki SX4 S-Cross 201...,YT-TPMS236;YT-TPMS242:\n Suzuki Jimny 1998-...
319,"[YT-TPMS236, YT-TPMS241]",Tesla Model S 2014-2022\nTesla Model X 2015-2022,YT-TPMS236;YT-TPMS241:\n Tesla Model S 2014...
320,"[YT-TPMS248, YT-TPMS249]",Cadillac Escalade Esv 2018,YT-TPMS248;YT-TPMS249:\n Cadillac Escalade ...


In [13]:
df_repeat_vehicle_explode = df_repeat_vehicle_groupby.explode('Part Number').fillna('')
df_merge_vehicle = pd.merge(df_all_repeat_number, df_repeat_vehicle_explode, on='Part Number', how='left').fillna('')
df_result_2 = df_merge_vehicle.groupby(['Part Number'], as_index=False, sort=False).agg({'重复车型':list})
df_result_2['重复车型'] = df_result_2['重复车型'].apply(lambda x: '\n'.join(x))
df_result_2

,Part Number,重复车型
0,YT-TPMS001,YT-TPMS001;YT-TPMS010:\n Buick Allure 2007-...
1,YT-TPMS002,YT-TPMS002;YT-TPMS007:\n Lexus Gs300 2005-2...
2,YT-TPMS003,YT-TPMS003;YT-TPMS014:\n Chrysler 300 2010-...
3,YT-TPMS004,YT-TPMS004;YT-TPMS237:\n Ford C-Max 2013-20...
4,YT-TPMS005,YT-TPMS004;YT-TPMS005;YT-TPMS030;YT-TPMS091:\n...
...,...,...
250,YT-TPMS251,
251,YT-TPMS252,YT-TPMS252;YT-TPMS254:\n Nissan Kicks 2022\...
252,YT-TPMS253,
253,YT-TPMS254,YT-TPMS252;YT-TPMS254:\n Nissan Kicks 2022\...


In [14]:
df_result = pd.merge(df_result_1, df_result_2, on='Part Number', how='left').fillna('')
df_result.rename(columns={'Vehicle':'车型'}, inplace=True)
df_result['计数-完整重复号码'] = ''
df_result['是否存在车型重复'] = ''
df_result['计数-完整重复号码'] = df_result['完整重复号码'].apply(lambda x: int(x.count(';'))+1 if x != '' else '-')
df_result['是否存在车型重复'] = df_result['重复号码'].apply(lambda x: '是' if x != '' else '否')
df_result

,Part Number,车型,重复号码,完整重复号码,重复车型,计数-完整重复号码,是否存在车型重复
0,YT-TPMS001,Buick Allure 2007-2008\nBuick Enclave 2008-201...,YT-TPMS001;YT-TPMS006;YT-TPMS008;YT-TPMS009;YT...,YT-TPMS001;YT-TPMS006;YT-TPMS008;YT-TPMS009;YT...,YT-TPMS001;YT-TPMS010:\n Buick Allure 2007-...,43,是
1,YT-TPMS002,Lexus CT200H 2011-2017\nLexus Gs300 2005-2006\...,YT-TPMS002;YT-TPMS007;YT-TPMS011;YT-TPMS018;YT...,YT-TPMS002;YT-TPMS003;YT-TPMS007;YT-TPMS011;YT...,YT-TPMS002;YT-TPMS007:\n Lexus Gs300 2005-2...,92,是
2,YT-TPMS003,Chrysler 300 2010-2021\nDodge Challenger 2010-...,YT-TPMS003;YT-TPMS014;YT-TPMS020;YT-TPMS076;YT...,YT-TPMS002;YT-TPMS003;YT-TPMS011;YT-TPMS013;YT...,YT-TPMS003;YT-TPMS014:\n Chrysler 300 2010-...,48,是
3,YT-TPMS004,Ford C-Max 2013-2018\nFord E-150 2010-2014\nFo...,YT-TPMS004;YT-TPMS005;YT-TPMS022;YT-TPMS023;YT...,YT-TPMS004;YT-TPMS005;YT-TPMS021;YT-TPMS022;YT...,YT-TPMS004;YT-TPMS237:\n Ford C-Max 2013-20...,18,是
4,YT-TPMS005,Ford Bronco 2021-2023\nFord Edge 2015-2023\nFo...,YT-TPMS004;YT-TPMS005;YT-TPMS021;YT-TPMS030;YT...,YT-TPMS004;YT-TPMS005;YT-TPMS021;YT-TPMS022;YT...,YT-TPMS004;YT-TPMS005;YT-TPMS030;YT-TPMS091:\n...,18,是
...,...,...,...,...,...,...,...
250,YT-TPMS251,Ram Promaster 1500 2022-2024\nRam Promaster 25...,,,,-,否
251,YT-TPMS252,Nissan Kicks 2021-2024\nNissan Sentra 2020-202...,YT-TPMS252;YT-TPMS254,YT-TPMS252;YT-TPMS254,YT-TPMS252;YT-TPMS254:\n Nissan Kicks 2022\...,2,是
252,YT-TPMS253,Nissan Ariya 2023-2024\nNissan Rogue 2021-2023,,,,-,否
253,YT-TPMS254,Nissan Kicks 2022\nNissan Sentra 2020-2023,YT-TPMS252;YT-TPMS254,YT-TPMS252;YT-TPMS254,YT-TPMS252;YT-TPMS254:\n Nissan Kicks 2022\...,2,是


In [15]:
wb = Workbook()
ws = wb.active
ws.title = "查重结果"
for r_idx, row in enumerate(dataframe_to_rows(df_result, index=False, header=True), 1):
    ws.append(row)
ws.column_dimensions['A'].width = 15
ws.column_dimensions['B'].width = 30
ws.column_dimensions['C'].width = 20
ws.column_dimensions['D'].width = 20
ws.column_dimensions['E'].width = 30
ws.column_dimensions['F'].width = 18
ws.column_dimensions['G'].width = 18
ws.freeze_panes = "A2"
font = Font(name='微软雅黑', size=9)
header_font = Font(name='微软雅黑', size=9, color="FFFFFF", bold=True)
fill = PatternFill(start_color="95B3D7", end_color="95B3D7", fill_type="solid")
alignment = Alignment(horizontal="center", vertical="center", wrap_text=True)
border = Border(
    left=Side(style='thin'),
    right=Side(style='thin'),
    top=Side(style='thin'),
    bottom=Side(style='thin')
)
for row in ws.iter_rows(min_row=2, max_col=ws.max_column, max_row=ws.max_row):
    for cell in row:
        cell.font = font
        cell.alignment = alignment
        cell.border = border
for cell in ws[1]:
    cell.font = header_font
    cell.fill = fill
    cell.alignment = alignment
    cell.border = border
ws.row_dimensions[1].height = 20
for row in range(2, ws.max_row + 1):
    ws.row_dimensions[row].height = 16.8

wb.save('./车型查重结果.xlsx')